# Фильтрационные методы отбора признаков в задачах Uplift-моделирования

В данном ноутбуке содержится исследование фильтрационных методов отбора признаков в задачах uplift-моделирования из библиотеки causalml

In [ ]:
!pip install scikit-uplift

In [ ]:
!pip install causalml

In [ ]:
!pip install catboost

In [ ]:
from sklift.datasets import fetch_lenta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

from causalml.inference.meta import BaseTLearner
from causalml.metrics import plot_gain, auuc_score
from causalml.feature_selection import FilterSelect

In [ ]:
SEED = 42

In [ ]:
dataset = fetch_lenta()
data, target, treatment = dataset.data, dataset.target, dataset.treatment

### Загрузка и предобработка данных

После проведения EDA пришли к следующим выводам:

1) В датасете присутсвует дисбаланс классов: число записей, поповших в тестовую выбоку (воздействие было) составляет 75%, а контрольная группа составляет 25%.

2) Так же в датасете есть дисбаланс таргета: всего лишь 10% покупателей совершили целевое дейсвтие, в то время как другие 90% этого дейсвтия не совершали.

3) В датасете есть пропуски - их нужно заполнить. Так как минимальное значение в датасете - 0, заполним пропсуски значением -100.

4) В датасете есть только один категориальный признак - пол клиента, в данном признаке есть пропуски. Для данного признака присвоим значение 1, если пол мужской, 0, если женский. Пропущенные значения заполним -1.

5) В датасете есть 30 полных дубликатов, от них необходимо избавиться.

6) В результате превичного анализа датаффрейма никакие признаки не были удалены.

Предобработаем данные с учетом этих выводов:

In [ ]:
# Преобразование признака gender
data['gender'] = data['gender'].map({'Male': 1, 'Female': 0})
data['gender'] = data['gender'].fillna(-1)

In [ ]:
# Заполнение пропусков
data = data.fillna(-100)

In [ ]:
data.isna().sum().sum()

np.int64(0)

In [ ]:
# Удаление явных дубликатов
data = data.drop_duplicates()

In [ ]:
# Разбиение на train, valid, test выборки

data['treatment'] = treatment
data['treatment'] = (data['treatment'] == 'test').astype(int)
data['target'] = target

train_val_idx, test_idx = train_test_split(data.index, test_size=0.2, random_state=SEED, stratify=data[['treatment', 'target']])

train_idx, val_idx = train_test_split(train_val_idx, test_size=0.25, random_state=SEED, stratify=data.loc[train_val_idx, ['treatment', 'target']])

X_train = data.drop(columns=['treatment', 'target']).loc[train_idx]
X_val = data.drop(columns=['treatment', 'target']).loc[val_idx]
X_test = data.drop(columns=['treatment', 'target']).loc[test_idx]

treatment_train = data['treatment'][train_idx]
treatment_val = data['treatment'][val_idx]
treatment_test = data['treatment'][test_idx]

y_train = target[train_idx]
y_val = target[val_idx]
y_test = target[test_idx]

### Отбор признаков

В качестве uplift-модели будем использовать T-learner, построенный на модели CatBoost:

In [ ]:
uplift_model = BaseTLearner(
    learner=CatBoostClassifier(
        loss_function='Logloss',
        verbose=False,
        random_state=SEED
    )
)

In [ ]:
def evaluate_auuc(model, X_train, y_train, t_train, X_test, y_test, t_test):

    model.fit(X_train, y_train, t_train)
    uplift_test = model.predict(X_test)

    df = pd.DataFrame({'uplift': uplift_test.squeeze(),
                       'y': y_test,
                       'treatment': t_test})

    return auuc_score(df, outcome_col='y', treatment_col='treatment')

Будем измерять качество модели на топ-k признаках

In [ ]:
results = []

In [ ]:
# # Посмотрим качество на всех признаках
auuc_base = evaluate_auuc(
    uplift_model,
    X_train,
    y_train,
    treatment_train,
    X_test,
    y_test,
    treatment_test
)

results.append({
    'filter': 'all_feats',
    'top_k': X_train.shape[1],
    'auuc': auuc_base
})

In [ ]:
filters = ['F', 'LR', 'KL', 'ED', 'Chi']
TOP_K = [20, 30, 40, 50, 60, 70, 80, 90, 100]

In [ ]:
results = []

for filter_name in filters:
    filter_method = FilterSelect()

    df = pd.concat([X_train, y_train, treatment_train], axis=1)\
           .rename(columns={'treatment': 'treatment_group_key'}).copy()

    X_features = X_train.columns.tolist()

    if filter_name == 'LR':
        df['const'] = 1.0
        X_features = ['const'] + X_features

    if filter_name == 'F':
        num_features = X_train.select_dtypes(include='number').columns.tolist()
        good_features = [
            f for f in num_features
            if X_train[f].nunique() > 2
            and X_train.loc[treatment_train == 1, f].var() > 1e-6
            and X_train.loc[treatment_train == 0, f].var() > 1e-6
        ]
        X_features = good_features

    if len(X_features) == 0:
        print(f'{filter_name} filter skipped: no valid features')
        continue

    try:
        f_imp = filter_method.get_importance(
            df,
            X_features,
            'response_att',
            filter_name,
            treatment_group=1,
            disp=False
        )
    except Exception as e:
        print(f'{filter_name} filter failed: {e}')
        continue

    for k in TOP_K:
        print(f'Method: {filter_name}, top-k: {k}')
        selected_features = f_imp['feature'].iloc[:k].tolist()
        if 'const' in selected_features:
            selected_features.remove('const')

        auuc = evaluate_auuc(
            uplift_model,
            X_train[selected_features],
            y_train,
            treatment_train,
            X_test[selected_features],
            y_test,
            treatment_test
        )

        results.append({
            'filter': filter_name,
            'top_k': k,
            'auuc': auuc
        })

/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:623: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["treatment_indicator"] = 0


F filter failed: wrong shape for coefs


/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:638: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["treatment_indicator"] = 0


LR filter failed: Singular matrix


/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:482: RuntimeWarning: All-NaN slice encountered
  for i_bin in range(np.nanmax(x_bin).astype(int) + 1):  # range(n_bins):
/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:482: RuntimeWarning: invalid value encountered in cast
  for i_bin in range(np.nanmax(x_bin).astype(int) + 1):  # range(n_bins):
/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:506: RuntimeWarning: All-NaN slice encountered
  min(n_bins, np.nanmax(x_bin).astype(int) + 1)
/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:506: RuntimeWarning: invalid value encountered in cast
  min(n_bins, np.nanmax(x_bin).astype(int) + 1)


Method: KL, top-k: 20
Method: KL, top-k: 30
Method: KL, top-k: 40
Method: KL, top-k: 50
Method: KL, top-k: 60
Method: KL, top-k: 70
Method: KL, top-k: 80
Method: KL, top-k: 90
Method: KL, top-k: 100


/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:482: RuntimeWarning: All-NaN slice encountered
  for i_bin in range(np.nanmax(x_bin).astype(int) + 1):  # range(n_bins):
/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:482: RuntimeWarning: invalid value encountered in cast
  for i_bin in range(np.nanmax(x_bin).astype(int) + 1):  # range(n_bins):
/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:506: RuntimeWarning: All-NaN slice encountered
  min(n_bins, np.nanmax(x_bin).astype(int) + 1)
/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:506: RuntimeWarning: invalid value encountered in cast
  min(n_bins, np.nanmax(x_bin).astype(int) + 1)


Method: ED, top-k: 20
Method: ED, top-k: 30
Method: ED, top-k: 40
Method: ED, top-k: 50
Method: ED, top-k: 60
Method: ED, top-k: 70
Method: ED, top-k: 80
Method: ED, top-k: 90
Method: ED, top-k: 100


/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:482: RuntimeWarning: All-NaN slice encountered
  for i_bin in range(np.nanmax(x_bin).astype(int) + 1):  # range(n_bins):
/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:482: RuntimeWarning: invalid value encountered in cast
  for i_bin in range(np.nanmax(x_bin).astype(int) + 1):  # range(n_bins):
/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:506: RuntimeWarning: All-NaN slice encountered
  min(n_bins, np.nanmax(x_bin).astype(int) + 1)
/usr/local/lib/python3.12/dist-packages/causalml/feature_selection/filters.py:506: RuntimeWarning: invalid value encountered in cast
  min(n_bins, np.nanmax(x_bin).astype(int) + 1)


Method: Chi, top-k: 20
Method: Chi, top-k: 30
Method: Chi, top-k: 40
Method: Chi, top-k: 50
Method: Chi, top-k: 60
Method: Chi, top-k: 70
Method: Chi, top-k: 80
Method: Chi, top-k: 90
Method: Chi, top-k: 100


In [ ]:
results

[{'filter': 'KL',
  'top_k': 20,
  'auuc': uplift    0.374668
  dtype: float64},
 {'filter': 'KL',
  'top_k': 30,
  'auuc': uplift    0.352126
  dtype: float64},
 {'filter': 'KL',
  'top_k': 40,
  'auuc': uplift    0.357165
  dtype: float64},
 {'filter': 'KL',
  'top_k': 50,
  'auuc': uplift    0.371625
  dtype: float64},
 {'filter': 'KL',
  'top_k': 60,
  'auuc': uplift    0.369194
  dtype: float64},
 {'filter': 'KL',
  'top_k': 70,
  'auuc': uplift    0.373232
  dtype: float64},
 {'filter': 'KL',
  'top_k': 80,
  'auuc': uplift    0.362531
  dtype: float64},
 {'filter': 'KL',
  'top_k': 90,
  'auuc': uplift    0.360133
  dtype: float64},
 {'filter': 'KL',
  'top_k': 100,
  'auuc': uplift    0.368281
  dtype: float64},
 {'filter': 'ED',
  'top_k': 20,
  'auuc': uplift    0.374668
  dtype: float64},
 {'filter': 'ED',
  'top_k': 30,
  'auuc': uplift    0.352126
  dtype: float64},
 {'filter': 'ED',
  'top_k': 40,
  'auuc': uplift    0.357165
  dtype: float64},
 {'filter': 'ED',
  'top_k'

In [ ]:
res_df = pd.DataFrame([{
    'filter': r['filter'],
    'top_k': r['top_k'],
    'auuc': r['auuc'].values[0]} for r in results])

In [ ]:
res_df.sort_values(by='auuc', ascending=False).head(6)

,filter,top_k,auuc
0,KL,20,0.374668
9,ED,20,0.374668
18,Chi,20,0.374668
14,ED,70,0.373232
23,Chi,70,0.373232
5,KL,70,0.373232


**Вывод:**

Фильтрационные методы позволяют оценить вклад каждого признака в различие treatment и control эффекта.

Можно заметить, что для всех фильтрационных методов метрика auuc на одинаковом значении top-k равна. И при этом самое максимальное значение auuc довольно низкое - 0.374668. При этом F и LR фильтры для поставленной задачи не рассчитываются с помощью библиотечной варицаии (а значит есть вероятность, что данные методы не подойдут для любого датасета). Потенциальные причины, почему F и LR филтры не сработали:


*   F-фильтр основан на F-тесте, который проверяет есть ли стат. значимая разность в отклике между trt и ctrl группами по указанному признаку. Этот метод требует числовые признаки с ненулевой дисперсией (в данном датасете признак с нулевой диспресией - gender). А также дисбаланс в группах trt/ctrl может привести к нестабильным оценкам дисперсии.
*   LR-фильтр использует логистическую регрессию с взаимодействием treatment и признака, чтобы оценить, насколько каждый признак влияет на различие отклика между treatment и control. А значит, данный метод чувствителен к мультиколлинеарности, при этом если в датасете есть признаки с низкой дисперсией, дубликаты или константные признаки, то метод приведется к ошмбке. Помимо этого, LR фильтр чувствителен к масштабу данных.

Таким образом, два данных метода нельзя запустить без дополнительного анализа признаков.

Кроме того, самое максимальное полученное значение auuc доволно низкое - 0.374668. И для каждого из фильтров значения AUUC совпадают для одинаковго числа отобранных фичей.




In [ ]:
num_features = X_train.select_dtypes(include='number').columns.tolist()
zero_var_features = [f for f in num_features if X_train[f].var() == 0]

print("Признаки с нулевой дисперсией:", zero_var_features)

Признаки с нулевой дисперсией: ['gender']
